In [1]:
import random
import os
import pickle
import json
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoProcessor
from transformers import DataCollatorWithPadding
from transformers import AutoModel, TrainingArguments, Trainer
# from transformers import RobertaTokenizer, RobertaModel

random.seed(42)
reshuffle_docs = False
percent_to_test = .1
percent_to_dev = .2
percent_to_train =  1 - percent_to_dev - percent_to_test

model_name = 'allenai/biomed_roberta_base'
dropout = .03



In [2]:
currentdir = os.getcwd() # ~/MeasEval/baselines

combopath_txt = os.path.join(currentdir, "../data/raw/combo/text/")
combopath_annot = os.path.join(currentdir, "../data/raw/combo/tsv/")

interimpath = os.path.join(currentdir, "../data/interim/")

docIds = []

if reshuffle_docs == True:
    combo_txt = {}
    for fn in os.listdir(combopath_txt):
        docIds.append(fn[:-4])
        path = combopath_txt+fn
        with open(path) as textfile:
                text = textfile.read()
                #[:-4] strips off the .txt to get the id
                combo_txt[fn[:-4]] = text

if reshuffle_docs == True:
    combo_annot = pd.DataFrame()
    for fn in os.listdir(combopath_annot):
        path = combopath_annot+fn
        file = pd.read_csv(path,delimiter='\t',encoding='utf-8')
        combo_annot = pd.concat([combo_annot, file],ignore_index=True)

    random.shuffle(docIds)

    n_doc = len(docIds)
    split_train = int(np.round(n_doc * percent_to_train))
    split_dev = split_train + int(np.round(n_doc * percent_to_dev))

    docs_train = docIds[:split_train]
    docs_dev = docIds[split_train:split_dev]
    docs_test = docIds[split_dev:]

    train_annot = combo_annot.loc[combo_annot['docId'].isin(docs_train)]
    dev_annot = combo_annot.loc[combo_annot['docId'].isin(docs_dev)]
    test_annot = combo_annot.loc[combo_annot['docId'].isin(docs_test)]

    # save data
    train_annot.to_csv(interimpath+'train_annot.csv')
    dev_annot.to_csv(interimpath+'dev_annot.csv')
    test_annot.to_csv(interimpath+'test_annot.csv')

    train_txt = {d: combo_txt[d] for d in docs_train}
    dev_txt = {d: combo_txt[d] for d in docs_dev}
    test_txt = {d: combo_txt[d] for d in docs_test}
    
    with open(interimpath+'train_txt.json','w') as f:
        json.dump(train_txt, f)
    with open(interimpath+'dev_txt.json','w') as f:
        json.dump(dev_txt, f)
    with open(interimpath+'test_txt.json','w') as f:
        json.dump(test_txt, f)

else:
    train_annot = pd.read_csv(interimpath+'train_annot.csv')
    dev_annot = pd.read_csv(interimpath+'dev_annot.csv')
    test_annot = pd.read_csv(interimpath+'test_annot.csv')

    with open(interimpath+'train_txt.json','r') as f:
        train_txt = json.load(f)
    with open(interimpath+'dev_txt.json','r') as f:
        dev_txt = json.load(f)
    with open(interimpath+'test_txt.json','r') as f:
        test_txt = json.load(f)




In [39]:
#todo EDA

ex_type = 'QUANT'
ex_num = 0
ex = train_annot.loc[train_annot['annotType'] == 'Quantity'].loc[ex_num]
ex_doc = ex['docId']

ex_txt = train_txt[ex_doc]
print(ex_txt)

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer(ex_txt))

ex_ent = train_txt[ex_doc][ex['startOffset']:ex['endOffset']]

print(ex_ent)

print(tokenizer(ex_ent))

The brief peak in Apectodinium, AOM and low salinity dinoflagellate cysts (Deflandrea) at 2617.4 m (Fig. 7) indicate a sporadic episode of surface water freshening/eutrophication before the CIE, which is best explained by an increase in regional precipitation due to its rapid nature. A coincident reduction in I. hiatus swamp conifers indicates possible disturbance of nearby coastal environments possibly from flooding (see Section 4.5). An associated reduction in δ13C may have been caused by stratification of the North Sea from an enhanced halocline, trapping 12C-enriched organic carbon at depth. This scenario may also explain the other peaks in Apectodinium at 2619.6 and 2614.7 m (although see Section 4.1).
{'input_ids': [0, 133, 4315, 4996, 11, 83, 13771, 1630, 179, 4031, 6, 83, 3765, 8, 614, 6641, 16838, 5917, 1116, 462, 1580, 890, 877, 19258, 5019, 36, 17425, 1245, 9891, 43, 23, 973, 1360, 4, 306, 475, 36, 44105, 4, 262, 43, 6364, 10, 33122, 3238, 9, 4084, 514, 21862, 700, 3509, 73,

RobertaConfig {
  "_name_or_path": "allenai/biomed_roberta_base",
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

NameError: name 'exampletxt' is not defined

In [ ]:
# todo: build models

config = AutoConfig.from_pretrained(model_name)
config

In [12]:

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# processor = AutoProcessor.from_pretrained(model_name)

config = AutoConfig.from_pretrained(model_name)

print(config)


RobertaConfig {
  "_name_or_path": "allenai/biomed_roberta_base",
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [13]:

model = AutoModel.from_config(config)

model.

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop